In [ ]:
import tensorflow as tf
from keras.datasets import imdb
from keras.utils import pad_sequences
import os
import numpy as np

In this dataset word encoding is like if a interger to a word is 3 it is referred as 3rd most common word in the data set.
Hence, num_words=Vocab_Size refers to load 88584 unique words in dataset

In [ ]:
VOCAB_SIZE = 88584

Max_Len = 250
Batch_Size = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:

print(train_data[0])
print(train_labels[i])

1
0
0
1
0
0
1
0
1
0
1
0
0
0
0


In [ ]:
print(len(train_data[0]))
print(len(train_data[1]))
print(len(train_data[2]))

218
189
141


In [ ]:
#In order to make our all reviews of same size
train_data = pad_sequences(train_data, Max_Len)
test_data = pad_sequences(test_data, Max_Len)

In [ ]:
print(len(train_data[0]))
print(len(train_data[1]))
print(len(train_data[2]))

250
250
250


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

In [ ]:
history = model.fit(train_data, train_labels, epochs = 10, validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 68s 96ms/step - loss: 0.4503 - acc: 0.7859 - val_loss: 0.3114 - val_acc: 0.8696
Epoch 2/10
625/625 [==============================] - 22s 36ms/step - loss: 0.2611 - acc: 0.9002 - val_loss: 0.2910 - val_acc: 0.8832
Epoch 3/10
625/625 [==============================] - 15s 24ms/step - loss: 0.1997 - acc: 0.9277 - val_loss: 0.2794 - val_acc: 0.8868
Epoch 4/10
625/625 [==============================] - 10s 17ms/step - loss: 0.1613 - acc: 0.9426 - val_loss: 0.3024 - val_acc: 0.8712
Epoch 5/10
625/625 [==============================] - 11s 17ms/step - loss: 0.1362 - acc: 0.9543 - val_loss: 0.3134 - val_acc: 0.8768
Epoch 6/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1119 - acc: 0.9631 - val_loss: 0.3884 - val_acc: 0.8832
Epoch 7/10
625/625 [==============================] - 8s 13ms/step - loss: 0.0966 - acc: 0.9694 - val_loss: 0.3481 - val_acc: 0.8820
Epoch 8/10
625/625 [==============================] - 9s 15ms/st

In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.5318 - acc: 0.8580
[0.5317881107330322, 0.8580399751663208]


In [ ]:
import keras.preprocessing

word_index = imdb.get_word_index() #mapping from which our old data was encoded

def encode_text(text):  #Covert Any text to proper preproccesed sentence according to loaded dataset
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], Max_Len)[0]



1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [ ]:
#decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
  return text[:-1]

In [ ]:
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  results = model.predict(pred)
  print(results[0])

In [ ]:
text = input("Enter Review: ")
result = np.argmax(predict(text)) #To convert into Int
if result >= 0.5:
  print("Positive Review")
else:
  print("Negitive Review")

Enter Review: It is so bad that i hate it
1/1 [==============================] - 0s 29ms/step
[0.14751723]
Negitive Review
